### 1. Imports

In [ ]:
import pandas as pd
import numpy as np
import os
import openai
import nltk
import random
import re
import ast
import time



from nltk.tokenize import sent_tokenize
nltk.download('punkt')


openai.api_key = # Enter your OPEN AI KEY

### 2. Data & Parameters

In [72]:
data = pd.read_csv('your_data.csv')

size_answers = 1200
sent_call = 100 #If split sentences, recommended size = 100, if unsplit recommended size = 50
split_sentences = "No"

number_questions = 9

topic_q1 = ["Keynote speakers","Breakout sessions", "Networking" , "The wellbeing space and inclusion", "The music and arts", "First Nations content", "other"]

topic_q2 = ["Time Management and Organization","Food and Beverage Options","Content of Keynotes and Breakout Sessions","Accessibility","Networking","Facilities","other"]
         
topic_q3 = ["Organization and Logistics", "Content", "Diversity and Inclusion", "other"]
         
topic_q4 = ["Connecting", "learn", "reflect & celebrate the work of the network","present and share knowledge", "other"]

topic_q5 = ["Networking","Learning","Understanding", "other"]

topic_q6 = ["Looking Forward to the Event","Expressing Gratitude","Networking","Learning New Ideas"]

topic_q7 = ["inspiring and motivating", "disorganized and not particularly useful", "need for more structured networking activities", "more professional development opportunities and in-depth discussions" , "more research to be presented / more failure stories to be shared / more diverse opportunities" , "more space to absorb the information presented / more shorter breaks." , "more space for networking" , "more advice on handling burnout and overextending oneself", "more visibility and representation of First Nations people", "more shared learnings between clinicians and between youth engagements","other"]

topic_q8 = ["Other"] # For theses questions it is easier to just read through

topic_q9 = ["Other"]  # For theses questions it is easier to just read through

all_topics= [topic_q1, topic_q2, topic_q3, topic_q4, topic_q5, topic_q6, topic_q7, topic_q8, topic_q9]


### 3. Defining functions

In [73]:
def remove_apostrophe(text):
    return text.replace("'", "")

def add_backslash(text):
    return text.replace("'", "\\'")

def flatten_list(nested_list):
    flat_list = []
    for item in nested_list:
        if isinstance(item, list):
            flat_list.extend(flatten_list(item))
        else:
            flat_list.append(item)
    return flat_list


In [74]:
def get_topics(data, all_topics, sent_call , split_sentences):
    
    if split_sentences == 'Yes':
        data['value'] = data['value'].apply(sent_tokenize)
        data = data.explode('value').reset_index(drop=True)
        data['value'] = data['value'].str.split('- ')
        data = data.explode('value').reset_index(drop=True)

    data = data[data['value'].str.len() > 3]
    data['value'] = data['value'].apply(add_backslash)

    dfs = []

    for value in data['variable'].unique():
        new_df = data[data['variable'] == value].copy()
        dfs.append(new_df)

    final_topics = []

    for i in range(len(dfs)):
        df = dfs[i]
        topics = all_topics[i]
        q = dfs[0]['variable']
        topics_output = []

        if len(df) > sent_call:
            sent = [f"{index + 1}. {sentence}" for index, sentence in enumerate(df['value'])]
            n = len(sent)
            sublists = [sent[i:i + sent_call] for i in range(0, n, sent_call)]

            for sublist in sublists:
                n_sub = len(sublist)
                p = f"Classify the topics in these '{n_sub}' sentences '{sublist}' using these topics '{topics}'. Print me the result for each sentence (just give me the number of the sentence and the corresponding topic, don't print the sentence again)"

                response = openai.Completion.create(
                        model="text-davinci-003",
                        prompt=p,
                        max_tokens=800,
                        temperature=0,
                        top_p=1.0,
                        frequency_penalty=0.0,
                        presence_penalty=0.0
                    )

                text = response['choices'][0]['text'].lstrip()
                lines = text.splitlines()
                topic_list = [line.split('. ', 1)[1].strip() for line in lines]

                for i in topic_list:
                    topics_output.append(i)

                # Introduce a pause of 2 seconds before making the next API call
                time.sleep(2)

        else:
            sent = [f"{index + 1}. {sentence}" for index, sentence in enumerate(df['value'])]
            n = len(sent)
            p = f"Classify the topics in these '{n}' sentences '{sent}' using these topics '{topics}'. Print me the result for each sentence (just give me the number of the sentence and the corresponding topic, don't print the sentence again)"

            response = openai.Completion.create(
                model="text-davinci-003",
                prompt=p,
                max_tokens=1500,
                temperature=0,
                top_p=1.0,
                frequency_penalty=0.0,
                presence_penalty=0.0
                )

            text = response['choices'][0]['text'].lstrip()
            lines = text.splitlines()
            print()
            topic_list = [line.split('. ', 1)[1].strip() for line in lines if len(line.split('. ', 1)) > 1]
            for i in topic_list:
                topics_output.append(i)

                # Introduce a pause of 2 seconds before making the next API call
            time.sleep(2)

        final_topics.append(topics_output)
        final_topics = flatten_list(final_topics)



    data["GPT_topics"] = final_topics
    data.to_excel('data_topics.xlsx', index=False)
    data.to_csv('data_topics.csv', index=False)
    
    return data



### 4. Results

In [69]:
topics = get_topics(data, all_topics, sent_call , split_sentences)

In [ ]:
topics